In [5]:
import cv2
import numpy as np


In [6]:
net = cv2.dnn.readNet('/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom_last.weights copy 5','/Users/amariofausta/Downloads/yolov3custom2.cfg')
with open("/Users/amariofausta/Documents/Kuliah/Semester 8/Perhitungan-Gerak-Jatuh-Bebas-menggunakan-algoritma-YOLOV3/classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [7]:
image = cv2.imread("/Users/amariofausta/Documents/Kuliah/Semester 8/Dataset/dataset/File 802.jpg")


In [8]:
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
outputs = net.forward(output_layers)
boxes = []
confidences = []
class_ids = []
iou_values = []
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > 0.5:
            center_x = int(detection[0] * image.shape[1])
            center_y = int(detection[1] * image.shape[0])
            width = int(detection[2] * image.shape[1])
            height = int(detection[3] * image.shape[0])
            left = int(center_x - width / 2)
            top = int(center_y - height / 2)
            
            boxes.append([left, top, width, height])
            confidences.append(float(confidence))
            class_ids.append(class_id)
for i in range(len(boxes)):
    for j in range(i + 1, len(boxes)):
        box1 = boxes[i]
        box2 = boxes[j]
        
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[0] + box1[2], box2[0] + box2[2])
        y2 = min(box1[1] + box1[3], box2[1] + box2[3])
        
        intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
        box1_area = box1[2] * box1[3]
        box2_area = box2[2] * box2[3]
        iou = intersection_area / float(box1_area + box2_area - intersection_area)
        
        iou_values.append(iou)
for i in range(len(boxes)):
    left, top, width, height = boxes[i]
    label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}, IoU: {iou_values[i]:.2f}"
    cv2.rectangle(image, (left, top), (left + width, top + height), (0, 255, 0), 2)
    cv2.putText(image, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
cv2.imshow("Object Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
